## 한국어 BERT의 마스크드 언어 모델 실습
사전 학습된 한국어 BERT 를 이용하여 마스크드 언어 모델을 실습해봅시다. 이번 실습을 위해서만이 아
니라 앞으로 사전 학습된 BERT 를 사용할 때는 transformers 라는 패키지를 자주 사용하게 됩니다. 실습
환경에 transformers 패키지를 설치해둡시다.


In [1]:
!pip install transformers

### 1. 마스크드 언어 모델과 토크나이저
transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT 는 이미 누군가가 학습해둔 모
델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다. 예를 들어서
A 라는 이름의 BERT 를 사용하는데, B 라는 이름의 BERT 의 토크나이저를 사용하면 모델은 텍스트를 제
대로 이해할 수 없습니다. A 라는 BERT 의 토크나이저는’ 사과’ 라는 단어를 36 번으로 정수 인코딩하는
반면에, B 라는 BERT 의 토크나이저는’ 사과’ 라는 단어를 42 번으로 정수 인코딩하는 등 단어와 맵핑되는
정수 정보 자체가 다르기 때문입니다.
klue/bert‐base 는 대표적인 한국어 BERT 입니다. klue/bert‐base 의 마스크드 언어 모델과 klue/bert‐
base 의 토크나이저를 로드해봅시다.

In [2]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

TFBertForMaskedLM.from_pretrained(‘BERT 모델 이름’) 을 넣으면 [MASK] 라고 되어있는 단어를 맞추
기 위한 마스크드 언어 모델링을 위한 구조로 BERT 를 로드합니다. 다시 말해서 BERT 를 마스크드 언어 모델 형태로 로드합니다. from_pt=True 는 해당 모델이 기존에는 텐서플로우가 아니라 파이토치로 학습
된 모델이었지만 이를 텐서플로우에서 사용하겠다라는 의미입니다.

AutoTokenizer.from_pretrained(‘모델 이름’) 을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크
나이저를 로드합니다.

In [3]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for prediction

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

### 2. BERT의 입력
'축구는 정말 재미있는 [MASK] 다’ 라는 임의의 문장이 있다고 해봅시다. 이를 마스크드 언어 모델의 입
력으로 넣으면, 마스크드 언어 모델은 [MASK] 의 위치에 해당하는 단어를 예측합니다. 마스크드 언어 모
델의 예측 결과를 보기위해서 klue/bert‐base 의 토크나이저를 사용하여 해당 문장을 정수 인코딩해봅시
다.

In [4]:
inputs = tokenizer('축구는 정망 재미있는 [MASK]다.', return_tensors = 'tf')

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


토크나이저로 변환된 결과에서 input_ids 를 통해 정수 인코딩 결과를 확인할 수 있습니다.

In [5]:
print(inputs['input_ids'])

tf.Tensor([[   2 4713 2259 1543 2087 6001 2259    4  809   18    3]], shape=(1, 11), dtype=int32)


토크나이저로 변환된 결과에서 token_type_ids 를 통해서 문장을 구분하는 세그먼트 인코딩 결과를 확
인할 수 있습니다.

In [6]:
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 11), dtype=int32)


현재의 입력은 문장이 두 개가 아니라 한 개이므로 여기서는 문장 길이만큼의 0 시퀀스를 얻습니다. 만약
문장이 두 개였다면 두번째 문장이 시작되는 구간부터는 1 의 시퀀스가 나오게 되지만, 여기서는 해당되
지 않습니다.

토크나이저로 변환된 결과에서 attention_mask 를 통해서 실제 단어와 패딩 토큰을 구분하는 용도인 어
텐션 마스크를 확인할 수 있습니다.

In [7]:
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 11), dtype=int32)


현재의 입력에서는 패딩이 없으므로 여기서는 문장 길이만큼의 1 시퀀스를 얻습니다. 만약 뒤에 패딩이
있었다면 패딩이 시작되는 구간부터는 0 의 시퀀스가 나오게 되지만, 여기서는 해당되지 않습니다. 좀 더
다양한 패턴의 입력은 뒤의 텍스트 분류, 개체명 인식, 질의 응답 실습에서 이어서 보겠습니다.

###3. [MASK] 토큰 예측하기
FillMaskPipeline 은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서
보여줍니다. FillMaskPipeline 에 우선 앞서 불러온 모델과 토크나이저를 지정해줍니다.


In [8]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Device set to use 0


이제 입력 문장으로부터 [MASK] 의 위치에 들어갈 수 있는 상위 5 개의 후보 단어들을 출력해봅시다.

In [9]:
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.8963507413864136,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.025957651436328888,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.01003396138548851,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924409583210945,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844234816730022,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [10]:
pip('어벤져스는 정말 재미있는 [MASK]다.')

[{'score': 0.83824223279953,
  'token': 3771,
  'token_str': '영화',
  'sequence': '어벤져스는 정말 재미있는 영화 다.'},
 {'score': 0.02827567793428898,
  'token': 568,
  'token_str': '거',
  'sequence': '어벤져스는 정말 재미있는 거 다.'},
 {'score': 0.01718931458890438,
  'token': 4665,
  'token_str': '드라마',
  'sequence': '어벤져스는 정말 재미있는 드라마 다.'},
 {'score': 0.014989699237048626,
  'token': 3758,
  'token_str': '이야기',
  'sequence': '어벤져스는 정말 재미있는 이야기 다.'},
 {'score': 0.009382622316479683,
  'token': 4938,
  'token_str': '장소',
  'sequence': '어벤져스는 정말 재미있는 장소 다.'}]

In [11]:
pip('나는 오늘 아침에 [MASK]에 출근을 했다.')

[{'score': 0.08012574166059494,
  'token': 3769,
  'token_str': '회사',
  'sequence': '나는 오늘 아침에 회사 에 출근을 했다.'},
 {'score': 0.06124085932970047,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '나는 오늘 아침에 에 출근을 했다.'},
 {'score': 0.0174866896122694,
  'token': 4345,
  'token_str': '공장',
  'sequence': '나는 오늘 아침에 공장 에 출근을 했다.'},
 {'score': 0.016131816431879997,
  'token': 5841,
  'token_str': '사무실',
  'sequence': '나는 오늘 아침에 사무실 에 출근을 했다.'},
 {'score': 0.015360787510871887,
  'token': 3671,
  'token_str': '서울',
  'sequence': '나는 오늘 아침에 서울 에 출근을 했다.'}]

## 한국어 BERT의 다음 문장 예측(Next Sentence Prediction)


###1. 다음 문장 예측 모델과 토크나이저
transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT 는 이미 누군가가 학습해둔 모
델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다.

In [12]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

TFBertForNextSentencePrediction.from_pretrained(‘BERT 모델 이름’) 을 넣으면 두 개의 문장이 이어
지는 문장 관계인지 여부를 판단하는 BERT 구조를 로드합니다.
AutoTokenizer.from_pretrained(‘모델 이름’) 을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크
나이저를 로드합니다.

In [14]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt = True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


###2. 다음 문장 예측하기
이제 TFBertForNextSentencePrediction 를 통해서 다음 문장을 예측해봅시다. 모델에 입력을 넣으면, 해
당 모델은 소프트맥스 함수를 지나기 전의 값인 logits 을 리턴합니다. 해당 값을 소프트맥스 함수를 통과
시킨 후 두 개의 값 중 더 큰 값을 모델의 예측값으로 판단하도록 더 큰 확률값을 가진 인덱스를 리턴하도
록 합니다.

In [16]:
# 이어지는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."

encoding = tokenizer(prompt, next_sentence, return_tensors='tf')
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 모델 : ', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 모델 :  [0]


최종 예측 레이블은 0 입니다. 이는 BERT 가 다음 문장 예측을 학습했을 당시에 실질적으로 이어지는 두
개의 문장의 레이블은 0. 이어지지 않는 두 개의 문장의 경우에는 레이블을 1 로 두고서 이진 분류로 학습
을 하였기 때문입니다. 이번에는 이어지지 않는 두 개의 문장으로 테스트해봅시다.

In [18]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"

encoding = tokenizer(prompt, next_sentence, return_tensors='tf')
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 모델 : ', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 모델 :  [1]
